In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

movies=pd.read_csv('../ml-latest/movies.csv')
rating=pd.read_csv('../ml-latest/ratings.csv')

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
movies['year']=movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies['year']=movies.year.str.extract('(\d\d\d\d)',expand=False)
movies['title']=movies.title.str.replace('(\(\d\d\d\d\))','')
movies['title']=movies.title.str.strip()
movies['genres']=movies.genres.str.split('|')

rating.drop('timestamp',1,inplace=True)

userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

user_record=movies[movies.title.isin(inputMovies['title'].tolist())]
user_record=pd.merge(user_record,inputMovies)
user_record.drop('year',1,inplace=True)

user_record

other_users=rating[rating['movieId'].isin(user_record['movieId'].tolist())]
other_users=other_users.groupby('userId')
other_users=sorted(other_users,key=lambda x: len(x[1]),reverse=True)

other_users=other_users[0:100]

import math
corr=dict()
for name,group in other_users:
    user_record=user_record.sort_values(by='movieId')
    group=group.sort_values(by='movieId')
    group_record=group[group.movieId.isin(user_record['movieId'].to_list())]
    user_rating=user_record['rating']
    group_rating=group_record['rating']
    n=len(group_rating)
    R_num=n*sum([i*j for i,j in zip(group_rating,user_rating)])-(sum(group_rating)*sum(user_rating))
    R_dx=n*sum([i**2 for i in group_rating])-pow(sum(group_rating),2)
    R_dy=n*sum([i**2 for i in user_rating])-pow(sum(user_rating),2)
    
    if R_dy!=0 and R_dx !=0:
        corr[name]=R_num/np.sqrt(R_dx*R_dy)
    else:
        corr[name]=0

sim_matrix=pd.DataFrame.from_dict(corr,orient='index')
sim_matrix.reset_index(inplace=True)
sim_matrix.columns=['userId','similarity index']

top_users=sim_matrix.sort_values(by='similarity index',ascending=False)
top_users=top_users.iloc[0:50,:]
top_users=top_users.merge(rating,left_on='userId',right_on='userId',how='inner')

top_users['weighted average']=top_users['rating']*top_users['similarity index']

new=top_users.groupby('movieId').sum()[['similarity index','weighted average']]

recommend=pd.DataFrame()
recommend['weighted average']=new['weighted average']/new['similarity index']

recommend.reset_index(inplace=True)

recommend.sort_values(by='weighted average',inplace=True)

movies[movies['movieId'].isin(recommend.head(10)['movieId'].tolist())]

,movieId,title,genres,year
1673,1733,Afterglow,"[Drama, Romance]",1997
2724,2809,Love Stinks,[Comedy],1999
4202,4296,Love Story,"[Drama, Romance]",1970
6783,6892,"Singing Detective, The","[Comedy, Drama, Musical, Mystery]",2003
11085,45981,Hollow Man II,"[Action, Horror, Sci-Fi, Thriller]",2006
12644,59306,Prom Night,"[Horror, Mystery, Thriller]",2008
14411,72043,Monsters vs Aliens: Mutant Pumpkins from Outer...,"[Animation, Comedy, Sci-Fi]",2009
21763,104952,I Was a Teenage Zombie,"[Comedy, Horror]",1987
21903,105390,"Rabid Grannies (Mémés cannibales, Les)","[Comedy, Horror]",1988
29159,133810,The Mad,"[Comedy, Horror, Thriller]",2007
